In [2]:
wrong_file = ['IS1003c.abssumm.xml',
 'IB4003.abssumm.xml',
 'ES2008c.abssumm.xml',
 'TS3004b.abssumm.xml',
 'TS3003b.abssumm.xml',
 'IS1005c.abssumm.xml',
 'TS3005b.abssumm.xml',
 'IS1000d.abssumm.xml',
 'TS3010a.abssumm.xml',
 'IB4005.abssumm.xml',
 'TS3003d.abssumm.xml',
 'ES2014d.abssumm.xml',
 'TS3004d.abssumm.xml',
 'IB4010.abssumm.xml',
 'TS3011d.abssumm.xml',
 'ES2006d.abssumm.xml',
 'ES2002c.abssumm.xml',
 'TS3012c.abssumm.xml',
 'TS3009b.abssumm.xml',
 'ES2007d.abssumm.xml',
 'IS1009c.abssumm.xml',
 'ES2015d.abssumm.xml',
 'ES2011a.abssumm.xml',
 'TS3005c.abssumm.xml',
 'ES2004a.abssumm.xml',
 'TS3003c.abssumm.xml',
 'ES2005a.abssumm.xml',
 'ES2014c.abssumm.xml',
 'TS3011c.abssumm.xml',
 'ES2010a.abssumm.xml',
 'TS3006d.abssumm.xml',
 'IS1004b.abssumm.xml',
 'ES2010b.abssumm.xml',
 'TS3012b.abssumm.xml',
 'IB4011.abssumm.xml',
 'IS1006c.abssumm.xml',
 'IS1001c.abssumm.xml',
 'ES2003b.abssumm.xml',
 'IS1004d.abssumm.xml']

In [3]:
import random
import json
from nltk.tokenize import sent_tokenize
dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'

good_file = []
for file in os.listdir(dir):
    if file in wrong_file:
        continue
    else:
        good_file.append(file)

NameError: name 'os' is not defined

In [76]:
'''
l =len(good_file)
result = {}
for i in range(5):
    
    train = []
    if i!=4:
        test = good_file[int(i*l/5):int((i+1)*l/5)]
    else:
        test = good_file[int((i)*l/5):]
        
    if i!=0:
        train.extend(good_file[:int((i)*l/5)])
        
    if i!=4:
        train.extend(good_file[int((i+1)*l/5):])
        
    #check 
    for f in train:
        if f in test:
            print(error)
        
    new_train = train[:int(0.85*len(train))]
    new_val = train[int(0.85*len(train)):]
    

    assert len(new_train)+len(new_val)+len(test)==l
    
    result[i] = {'train':new_train,'val':new_val,'test':test}
    
with open('data-resplit/split.json', 'w') as f:
    json.dump(result, f) 
'''    

In [4]:
import xml.etree.ElementTree as ET
import nltk
import os
from collections import defaultdict

dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/extractive/'

link_map = defaultdict(list)
for file in os.listdir(dir):
    if 'summlink' in file and ".#" not in file:
        tree = ET.parse(dir+file)
        root = tree.getroot()
        for content in root.findall('summlink'):
            val = None
            for child in content:
                if val == None:
                    val = child.attrib['href'].split("(")[1].replace(")","")
                else:
                    key = child.attrib['href'].split("(")[1].replace(")","")
            link_map[key].append(val)

In [5]:
def get_da(left,right):
    tmp_dir =  '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/words/'
    file = ".".join(left.split(".")[0:2])+".words"
    tree = ET.parse(tmp_dir+file+".xml")
    root = tree.getroot()
    left = int(left.replace(file,""))
    time = float('inf')
    
    if right ==None:
        right = left
    else:
        right = int(right.replace(file,""))
    da = []
    for content in root.findall('w'):
        id = int(content.attrib['{http://nite.sourceforge.net/}id'].replace(file,""))
        if id>= left and id <=right:
            time = min(time,float(content.get('starttime')))
            value = content.text
            #key = content.get('starttime')
            da.append(value.replace("_",""))
    return " ".join(da),time


In [6]:
def get_drda(file_name, da):
    tmp_dir =  '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/dialogueActs/'
    tree = ET.parse(tmp_dir+".".join(da.split(".")[0:3])+".xml")
    root = tree.getroot()
    for content in root.findall('dact'):
        id = content.attrib['{http://nite.sourceforge.net/}id']
        if id == da:
            drda_type = None
            for node in content:
                if 'role' in node.attrib:
                    drda_type = node.attrib['href'].split("_")[-1].split(")")[0]
                if 'role' not in node.attrib:
                    href = node.attrib['href']
                    left,right =None,None
                    for val in href.split("#")[1].split(".."):
                        if left==None:
                            left = val.replace("id(","").replace(")","")
                        else:
                            right = val.replace("id(","").replace(")","")
            
            return left,right, drda_type
            



In [7]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    all_sum = []
    f_name = []
    
    for cross_split in range(5):
        tfidf_input = []
        tfidf = None
        for mode in ["train","val","test"]:
            try:
                os.mkdir('pair_ntm_sentence_'+str(cross_split+1))
            except:
                pass
        
            try:
                os.mkdir('pair_ntm_sentence_'+str(cross_split+1)+"/"+mode)
            except:
                pass
            
                
            pos_example, neg_example = 0,0
            r = []
            for file in os.listdir("data_nice_split/"+'pair_ntm_sentence_'+str(cross_split+1)+"/"+mode):
                r.append(file.split(".")[0]+'.abssumm.xml')

            if tfidf_input == []:
                for file in r:
                    _,_, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                    tfidf_input.extend(corpus)
                with open('pair_ntm_sentence_'+str(cross_split+1)+"/"+"tfidf.json", 'w') as f:
                    json.dump(tfidf_input, f)               
                vectorizer = TfidfVectorizer()
                tfidf = vectorizer.fit(tfidf_input)
                    
            for file in r:
                dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
                tree = ET.parse(dir+file)
                root = tree.getroot()

                
                transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                
                corpus_tfidf = tfidf.transform(corpus)
                text = []
                sent_dict = defaultdict(str)
                flag = True
                group = 0
                
                drda = []
                stats = []
                
                for content in root.findall('decisions'):
                    for sentence in content.findall('sentence'):
                        summary = sentence.text.replace("\n","")
                        all_summaries.append(summary)
                        num_sent = 0
                        group += 1
                        f_name.append(file)
                        decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                        ext_link = link_map[decision_id]
                        if ext_link ==[]:
                            flag = False
                        else:
                            for i in ext_link:
                                left,right,drda_type = get_drda(file, i)
                                da,time = get_da(left,right)
                                das = sent_detector.tokenize(da)
                                for comp in das:
                                    if search_trans(transcript, key, comp)==None:
                                        #print(comp)
                                        #print(transcript)
                                        comp = comp.replace(". ","")
                                    if 'just kinetic then' in comp:
                                        comp = 'just kinetic then'

                                    pos, sentence, s_time = search_trans(transcript, key, comp)
                                    #print(s_time,time)
                                    if sentence =="":
                                        continue
                                    if pos not in sent_dict:
                                        if len(sentence.split())<3:
                                            continue
                                        context_s = corpus[find_context(tfidf.transform([sentence]).toarray(),corpus_tfidf.toarray())]
                                        text.append((s_time,sentence,group,pos,context_s,summary))
                                        stats.append(group)
                                        drda.append(sentence)
                                        num_sent +=1
                                    sent_dict[pos] = sentence
                        avg.append(num_sent)
                text.sort(key = lambda x: x[0]) 
                
                #pos/neg calculation
                c = list(dict(Counter(stats)).values())
                for i in range(len(c)):
                    if c[i]>1:
                        pos_example += c[i]*(c[i]-1)/2.0
                    for j in range(i+1,len(c)):
                        neg_example += c[i]*c[j]
                                       
                result = []
                for i in range(len(text)):
                    #sentence_pos,time, sentence, group, pos over passage, context sentence, drdas,summaries
                    result.append((i,text[i][0],text[i][1],text[i][2],text[i][3], text[i][4]," ".join(drda),text[i][5]))
                for i in text:
                    with open('pair_ntm_sentence_'+str(cross_split+1)+"/"+mode+"/"+file.split(".")[0]+".json", 'w') as f:
                        json.dump(result, f)

    print(sum(avg)/len(avg))
generate_data()

NameError: name 'generate_transcript' is not defined

In [8]:
##final all together data
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    f_name = []
    
    for cross_split in range(1):
        tfidf_input = []
        tfidf = None
        for mode in ["train","val","test"]:
            try:
                os.mkdir('pair_ntm_sentence_'+str(cross_split+1))
            except:
                pass
        
            try:
                os.mkdir('pair_ntm_sentence_'+str(cross_split+1)+"/"+mode)
            except:
                pass
            
            r = []
            for file in os.listdir("data_nice_split/"+'pair_ntm_sentence_'+str(cross_split+1)+"/"+mode):
                r.append(file.split(".")[0]+'.abssumm.xml')

            if tfidf_input == []:
                for file in r:
                    _,_, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                    tfidf_input.extend(corpus)
                with open('pair_ntm_sentence_'+str(cross_split+1)+"/"+"tfidf.json", 'w') as f:
                    json.dump(tfidf_input, f)               
                vectorizer = TfidfVectorizer()
                tfidf = vectorizer.fit(tfidf_input)
                    
            for file in r:
                dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
                tree = ET.parse(dir+file)
                root = tree.getroot()

                
                transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                
                corpus_tfidf = tfidf.transform(corpus)
                text = []
                sent_dict = defaultdict(str)
                flag = True
                group = 0
                
                drda = []
                stats = []
                all_summaries = []
                
                
                for content in root.findall('decisions'):
                    for sentence in content.findall('sentence'):
                        summary = sentence.text.replace("\n","")
                        all_summaries.append(summary)
                        num_sent = 0
                        
                pos = 0
                for i in key:
                    sentence = []
                    for word in transcript[i][2:].split():
                        if word.lower() in utterance:
                            continue
                        else:
                            sentence.append(word)
                    if len(sentence) < 3:
                        continue
                    else:
                        sentence = " ".join(sentence)
                    s_time = i
                    context_s = corpus[find_context(tfidf.transform([sentence]).toarray(),corpus_tfidf.toarray())]
                    text.append((s_time,sentence,1,pos,context_s,""))
                    pos += 1
                         
                result = []
                for i in range(len(text)):
                    #da_pos,time, sentence, group, pos over passage, context sentence, drdas,summaries
                    result.append((i,text[i][0],text[i][1],text[i][2],text[i][3], text[i][4],"",text[i][5]))

                with open('pair_ntm_sentence_'+str(cross_split+1)+"/"+mode+"/"+file.split(".")[0]+".json", 'w') as f:
                    json.dump({"transcript":result,"summary":" ".join(all_summaries)}, f)

generate_data()


NameError: name 'generate_transcript' is not defined

In [9]:
import re
import os
import collections
import xml.etree.ElementTree as ET
import string
import html
import nltk.data


sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

utterance = ['uh','hmm','um','mm','huh','mm-hmm']

def generate_transcript(file_list,speakers):
    mydict = {}
    corpus = []
    endtime = {}
    dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/words/'
    for n in range(len(file_list)):
        tree = ET.parse(dir+file_list[n])
        root = tree.getroot()
        token = []
        time = []

        for content in root.findall('w'):
            value = content.text
            key = content.get('starttime')
            #if value.lower() in utterance:
                #continue
            if len(token)>=1 and token[-1] in string.punctuation and value in string.punctuation:
                continue
            #if value.lower() in utterance:
               # continue
            if key == None:
                continue
            token.append(value)
            time.append(key)

        sentences = sent_detector.tokenize(" ".join(token))
        count = 0
        for sentence in sentences:
            mydict[float(time[count])] = speakers[n] +":"+sentence.replace("_","")
            corpus.append(sentence.replace("_",""))
            prev_count = count
            count += len(sentence.split(" "))
            endtime[float(time[prev_count])] = time[count-1]



    mydict_sort = sorted(mydict)
    remove_lst = []
    for key in mydict_sort:
        sentence = mydict[key]
        sentence = " ".join([word.lower() for word in sentence[2:].split() if word.lower() not in utterance])
        if not any(c.isalpha() for c in sentence):
            remove_lst.append(key)
            del mydict[key]
    
    for i in remove_lst:
        mydict_sort.remove(i)
        
    new_corpus = []
    for key in mydict_sort:
        new_corpus.append(mydict[key])
        

    return mydict,mydict_sort,new_corpus,endtime

    '''
    insert_lines = []

    for i in mydict_sort:
        sentence = mydict[i].split(':')[0]
    '''
        


files = collections.defaultdict(list)
speaker  =collections.defaultdict(list)
dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/words/'
for f in os.listdir(dir):
    files[f.split('.')[0]].append(f)
    speaker[f.split('.')[0]].append(f.split('.')[1])

In [10]:
def search_trans(transcript, key, da,time):
    #print(da)
    #print("___")
    
    #print(transcript)
    #print(da)
    count = 0 
    lookup = []
    for item,i in enumerate(key):
        sentence = transcript[i][2:]
        
        if da.lower() in sentence.lower():
            #print(sentence)
            diff = round(abs(i-time),2)
            lookup.append((diff,i,item))
 
    lookup.sort(key=lambda x:x[0])
    
    clean_s = []
    if lookup==[]:
        print(transcript,da)
        
    for word in transcript[lookup[0][1]][2:].split():
        if word.lower() in utterance:
            continue
        else:
            clean_s.append(word.lower())
    if len(clean_s)<2:
        return count,"",""
    return lookup[0][2],transcript[lookup[0][1]][:2]+" "+" ".join(clean_s),lookup[0][1]


In [117]:
import json
sentence_summary_map = {}
def ntm_data(split_num=0,dtype='test'):
    data = []
    all_sum = []
    f_name = []

    true_label = []
    train_sentence = []
    r = []

    f = open ('data-resplit/split.json', "r")
  

    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]


    alldata = []
    posn=0
    negn = 0
    for file in r:        
        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        transcript, key,corpus,endtime = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])

        sent_dict = defaultdict(str)
        flag = True
        
        selected = []
        summaries = []
        alldas = []
        allsavedkey = []
        
        tmp_c = 0
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):
                summary = sentence.text.replace("\n","")
                all_sum.append(summary)
                f_name.append(file)
                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                ext_link = link_map[decision_id]

                if ext_link ==[]:
                    flag = False
                else:
                    for i in ext_link:
                        left,right,_ = get_drda(file, i)
                        da,time = get_da(left,right)
                        alldas.append(da)
                        das = sent_detector.tokenize(da)

                        for comp in das:
                            if len(comp.split())<3:
                                continue
                            if 'just kinetic then' in comp:
                                comp = 'just kinetic then'
                            pos, sentence, savedkey = search_trans(transcript, key, comp,time)
                            if sentence =="":
                                print(comp)
                                continue
                            if sentence not in selected:
                                tmp_c +=1
                                selected.append(sentence)
                                allsavedkey.append(savedkey)
                                #sentence_summary_map[sentence] = summary
                                #tmp.append(sentence)
        kk = []
        file_train_sentence = []
        file_true_label = []
        posn = 0
        for i in key:
            sentence = transcript[i]
            ori = sentence
            sentence = " ".join([word.lower() for word in sentence[2:].split() if word.lower() not in utterance])
            #print(sentence)
            if any(c.isalpha() for c in sentence):
                file_train_sentence.append(ori[0:2]+" "+sentence)
                if i in allsavedkey:
                    file_true_label.append(True)
                    posn +=1
                else:
                    file_true_label.append(False)
                    negn +=1

        true_label.append(file_true_label)
        train_sentence.append(file_train_sentence)
        assert len(file_true_label) == len(file_train_sentence)
    
    assert len(true_label) == len(train_sentence)
    with open('sentence_extraction_train/'+str(split_num)+"_seq_model_"+dtype+'.json', 'w') as f:
        json.dump([train_sentence,true_label], f)     
    #with open('sentence_extraction_train/'+str(split_num)+"_"+dtype+"_summary"+'.json', 'w') as f:
        #json.dump(all_sum, f)        

    ''''
    for i in text:
        file1 = open("meetingsentence_lines_test.txt", "a")  # append mode
        file1.write(i+"\n")
        file1.close()
    '''
    #f = open("sentence_to_summary.json","w")
    #json.dump(sentence_summary_map, f)
    
for split_num in [2,3,4]:
    for dtype in ['train','val','test']:
        ntm_data(split_num=split_num, dtype=dtype)

In [133]:
f = open ('test_15_sent.json', "r")


train = json.loads(f.read())


In [134]:
def my_f1_score(y_true, y_pred):
    a = 0
    for i in y_pred:
        if i in y_true:
            a +=1
            
    p = a/len(y_true)
    r = a/len(y_pred)
    try:
        val = 2*p*r/(p+r)
    except:
        val = 0
    return val

In [136]:
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json


files = collections.defaultdict(list)
speaker  =collections.defaultdict(list)
dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/words/'
for f in os.listdir(dir):
    files[f.split('.')[0]].append(f)
    speaker[f.split('.')[0]].append(f.split('.')[1])
    

def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data(split_num=1,dtype='test'):
    avg = []
    data = []
    all_sum = []
    f_name = []
    count = 0 
 
    with open(os.path.join("sentence_extraction_train",'1_seq_model_test.json'), 'r') as f:
        seq = json.loads(f.read())
        
    f = open ('data-resplit/split.json', "r")  

    pos_example,neg_example =0,0
    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]

    tfidf_input = []

    for file in data['train']:

        _,_, corpus,_ = generate_transcript(files[file.split(".")[0]],speaker[file.split(".")[0]])
        
        tfidf_input.extend(corpus)

    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit(tfidf_input)
    f1score = []
    for file in r:
        select_sentence = train[count]
        count +=1
        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        
        transcript, key, corpus,_ = generate_transcript(files[file.split(".")[0]],speaker[file.split(".")[0]])
        
        
        corpus_tfidf = tfidf.transform(corpus)
        text = []
        sent_dict = defaultdict(str)
        flag = True
        group = 0

        drda = []
        stats = []

        all_summaries,inputs = [],[]

    
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):
                tmp = []
                summary = sentence.text.replace("\n","")
                all_summaries.append(summary)
                f_name.append(file)
                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                ext_link = link_map[decision_id]
                if ext_link ==[]:
                    flag = False
                else:
                    for i in ext_link:
                        left,right,drda_type = get_drda(file, i)
                        da,time = get_da(left,right)
                        das = sent_detector.tokenize(da)
                        for comp in das:
                            if len(comp.split())<3:
                                continue
                            if 'just kinetic then' in comp:
                                comp = 'just kinetic then'
                            if search_trans(transcript, key, comp,time)==None:
                                #print(comp)
                                #print(transcript)
                                comp = comp.replace(". ","")


                            pos, sentence, s_time = search_trans(transcript, key, comp,time)
                            tmp.append(s_time)
                             
                        
                     
                inputs.append(tmp)               
        #assert sum(seq[1][count-1]) ==len(set([item for sublist in inputs for item in sublist]))
        
        fast_rl_group = []
        for _ in range(len(inputs)+1):
            fast_rl_group.append([])

        ttt = []
        for item,k in enumerate(key):    
            if item in select_sentence:
                group = -1
                for gidx,g in enumerate(inputs):
                    if k in g:
                        group = gidx
                fast_rl_group[group].append([transcript[k],k,item])
            for gidx,g in enumerate(inputs):
                if k in g:
                    ttt.append(item)

        if inputs==[]:
            f1score.append(0)
        else:
            f1score.append(my_f1_score(list(set(ttt)), select_sentence))

        aseertnum2 = 0
        
        fast_rl_group = [i for i in fast_rl_group if i!=[]]
        
        for groupidx,group in enumerate(fast_rl_group):
            for sentence,time,pos in group:
                if len(sentence.split())<3:
                    continue
                context_s = corpus[find_context(tfidf.transform([sentence]).toarray(),corpus_tfidf.toarray())]
                if groupidx>=len(all_summaries):
                    tmpsum=""
                else:
                    tmpsum = all_summaries[groupidx]
                text.append((time,sentence,groupidx,pos,context_s,tmpsum))
                if groupidx!=len(inputs):
                    aseertnum2 +=1
                stats.append(groupidx)
                drda.append(sentence)

        #assert aseertnum2 ==assertnum1
        text.sort(key = lambda x: x[0]) 

        #pos/neg calculation

        c = list(dict(Counter(stats)).values())
        for i in range(len(c)):
            if c[i]>1:
                pos_example += c[i]*(c[i]-1)/2.0
            for j in range(i+1,len(c)):
                neg_example += c[i]*c[j]

        result = []
        for i in range(len(text)):
            #sentence_pos,time, sentence, group, pos over passage, context sentence, drdas,summaries
            result.append((i,text[i][0],text[i][1],text[i][2],text[i][3], text[i][4]," ".join(drda),text[i][5]))
        
        
        
        for i in text:
            with open('new_cluster_data'+"/"+dtype+"_15_sent/"+file.split(".")[0]+".json", 'w') as f:
                json.dump(result, f)
        print(sum(f1score)/len(f1score))
    print(pos_example,neg_example)
    print(sum(f1score)/len(f1score))

generate_data()

0.21428571428571427
0.13655462184873948
0.12133944486885662
0.09100458365164246
0.09385629850026134
0.15397115784112686
0.16558872352768855
0.16572346642006083
0.19175419237338742
0.18507877313604867
0.18643524830549882
0.17847473518913148
0.20577155043099316
0.2139307254002079
0.19966867704019406
0.19343938472518193
0.18859654510082483
0.17811895926189011
0.17576182105512397
0.18602134904998682
0.1883676713641331
1651.0 554
0.1883676713641331


In [108]:
len_summ = []
len_input = []
def generate_data(split_num=1,dtype='test'):
    avg = []
    data = []
    all_sum = []
    f_name = []
    count = 0 
 
    f = open ('data-resplit/split.json', "r")  

    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]+data['val']+data['train']

    file_summary = {}
    file_summary_order = []
    all_text = []
    for file in r:

        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        
        transcript, key, corpus,_ = generate_transcript(files[file.split(".")[0]],speaker[file.split(".")[0]])
        

        
        flag = True
        group = 0

        drda = []
        stats = []

        tmp_sum = []
        
        
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):
                text = []
                sent_dict = defaultdict(str)
                
                
                summary = sentence.text.replace("\n","")
                tmp_sum.append(summary)
                file_summary_order.append(summary)
                len_summ.append(len(summary.split()))

                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                #print(decision_id)
                ext_link = link_map[decision_id]

                if ext_link ==[]:
                    flag = False
                else:
                    for i in ext_link:
                        left,right,_ = get_drda(file, i)
                        da,time = get_da(left,right)
                        das = sent_detector.tokenize(da)
                        for comp in das:
                            if len(comp.split())<3:
                                continue
                            if 'just kinetic then' in comp:
                                comp = 'just kinetic then'
                            if search_trans(transcript, key, comp,time)==None:
                                #print(comp)
                                #print(transcript)
                                comp = comp.replace(". ","")

                            pos, sentence, s_time = search_trans(transcript, key, comp,time)

                            if sentence =="":
                                print(comp)
                                continue

                            if pos not in sent_dict:
                                text.append(sentence[3:])
                            sent_dict[pos] = sentence
        
                all_text.append(" ".join(text))  
                len_input.append(len(" ".join(text).split()))

        file_summary[file.split(".")[0]] = " ".join(tmp_sum)  

        #len_summ.append(len(" ".join(tmp_sum).split()))
        #len_input.append(len(" ".join(text).split()))

    assert len(all_text)==len(file_summary_order)
    
    assert len(file_summary)==len(r)
    
    if dtype=='test':
        with open('bart_data/test_sum_gold.json', 'w') as f:
            json.dump(file_summary, f)     

    '''
    for i in range(len(file_summary)):
        file1 = open("bart_data/cluster_"+str(split_num)+"_"+dtype+".txt.src","a")
        file1.write(all_text[i]+"\n")
        file2 = open("bart_data/cluster_"+str(split_num)+"_"+dtype+".txt.tgt.tagged","a")
        file2.write(file_summary_order[i]+"\n")

    '''
generate_data()                    

In [93]:
min(len_input)

0

In [91]:
min(len_summ)

3

In [ ]:
len_summ = []
len_input = []
def generate_data(split_num=1,dtype='test'):
    avg = []
    data = []
    all_sum = []
    f_name = []
    count = 0 
 
    f = open ('data-resplit/split.json', "r")  

    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]

    file_summary = {}
    file_summary_order = []
    all_text = []
    for file in r:

        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        
        transcript, key, corpus,_ = generate_transcript(files[file.split(".")[0]],speaker[file.split(".")[0]])
        

        
        flag = True
        group = 0

        drda = []
        stats = []

        tmp_sum = []
        
        text = []
        sent_dict = defaultdict(str)
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):

                summary = sentence.text.replace("\n","")
                tmp_sum.append(summary)

                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                #print(decision_id)
                ext_link = link_map[decision_id]

                if ext_link ==[]:
                    flag = False
                else:
                    for i in ext_link:
                        left,right,_ = get_drda(file, i)
                        da,time = get_da(left,right)
                        das = sent_detector.tokenize(da)
                        for comp in das:
                            if len(comp.split())<3:
                                continue
                            if 'just kinetic then' in comp:
                                comp = 'just kinetic then'
                            if search_trans(transcript, key, comp,time)==None:
                                #print(comp)
                                #print(transcript)
                                comp = comp.replace(". ","")

                            pos, sentence, s_time = search_trans(transcript, key, comp,time)

                            if sentence =="":
                                print(comp)
                                continue

                            if pos not in sent_dict:
                                text.append(sentence[3:])
                            sent_dict[pos] = sentence
        
                

        file_summary[file.split(".")[0]] = " ".join(tmp_sum)  
        file_summary_order.append(" ".join(tmp_sum))
        all_text.append(" ".join(text))  

        #len_summ.append(len(" ".join(tmp_sum).split()))
        #len_input.append(len(" ".join(text).split()))

    assert len(all_text)==len(file_summary_order)
    
    assert len(file_summary)==len(r)
    
    if dtype=='test':
        with open('bart_data_long/test_sum_gold.json', 'w') as f:
            json.dump(file_summary, f)     

    for i in range(len(file_summary)):
        file1 = open("bart_data_long/cluster_"+str(split_num)+"_"+dtype+".txt.src","a")
        file1.write(all_text[i]+"\n")
        file2 = open("bart_data_long/cluster_"+str(split_num)+"_"+dtype+".txt.tgt.tagged","a")
        file2.write(file_summary_order[i]+"\n")

                  
generate_data()                    

In [210]:
len_summ = []
len_input = []
def generate_data(split_num=4,dtype='train'):
    avg = []
    data = []
    all_sum = []
    f_name = []
    count = 0 
 
    f = open ('data-resplit/split.json', "r")  

    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]

    file_summary = {}
    file_summary_order = []
    all_text = []
    for file in r:

        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        
        transcript, key, corpus,_ = generate_transcript(files[file.split(".")[0]],speaker[file.split(".")[0]])
        

        
        flag = True
        group = 0

        drda = []
        stats = []

        tmp_sum = []
        
        text = []
        sent_dict = defaultdict(str)
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):

                summary = sentence.text.replace("\n","")
                tmp_sum.append(summary)

                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
        
                

        file_summary[file.split(".")[0]] = " ".join(tmp_sum)  
        file_summary_order.append(" ".join(tmp_sum))
        all_text.append(" ".join(corpus))  

        len_summ.append(len(" ".join(tmp_sum).split()))
        len_input.append(len(" ".join(corpus).split()))

    assert len(all_text)==len(file_summary_order)
    
    assert len(file_summary)==len(r)
    
    if dtype=='test':
        with open('bart_data_all/test_sum_gold'+str(split_num)+'.json', 'w') as f:
            json.dump(file_summary, f)     

    for i in range(len(file_summary)):
        file1 = open("bart_data_all/cluster_"+str(split_num)+"_"+dtype+".txt.src","a")
        file1.write(all_text[i]+"\n")
        file2 = open("bart_data_all/cluster_"+str(split_num)+"_"+dtype+".txt.tgt.tagged","a")
        file2.write(file_summary_order[i]+"\n")

                  
generate_data()                    

In [16]:
sum(len_input)/len(len_input)

5518.985507246377

In [17]:
max(len_input)

11115

In [33]:
#seed
from collections import defaultdict
len_summ = []
len_input = []
verb_count = defaultdict(int)
def generate_data(split_num=1,dtype='train'):
    avg = []
    data = []
    all_sum = []
    f_name = []
    count = 0 
 
    f = open ('data-resplit/split.json', "r")  

    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]

    file_summary = {}
    file_summary_order = []
    tmp_sum = []
    tmp_input = []
    all_text = []
    
    
    for file in r:

        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        
        transcript, key, corpus,_ = generate_transcript(files[file.split(".")[0]],speaker[file.split(".")[0]])
        

                    

        
        flag = True
        group = 0

        drda = []
        stats = []

        
        
        text = []
        sent_dict = defaultdict(str)
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):

                summary = sentence.text.replace("\n","")
                tmp_sum.append(summary)
                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                #print(decision_id)
                ext_link = link_map[decision_id]
                '''
                if ext_link ==[]:
                    flag = False
                else:
                    for i in ext_link:
                        left,right,_ = get_drda(file, i)
                        da,time = get_da(left,right)
                        das = sent_detector.tokenize(da)
                        for comp in das:
                            if 'just kinetic then' in comp:
                                comp = 'just kinetic then'
                            if len(comp.split())<3:
                                continue
                            pos, sentence, savedkey = search_trans(transcript, key, comp,time)
                            tmp_input.append(sentence)
                            s = sentence.split(" ")
                            s_pos_tag = pos_tag(s)
                            s = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else lemmatizer.lemmatize(i) for i,j in s_pos_tag]
                            for w,tag in zip(s,s_pos_tag):
                                if tag[1][0].lower() in ['v']:
                                    verb_count[w] +=1
                            
                '''
    with open('train_'+str(split_num)+'_summary_input.json', 'w') as f:
        json.dump(tmp_sum, f)
for i in range(5):                  
    generate_data(i)                    

In [166]:
import json
file = open("/Users/heatherzheng/Downloads/word_0_new.json")
seed=json.load(file)

In [167]:
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk import pos_tag
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()

stop_words = stopwords.words('english')
import json
import csv
sentence_summary_map = {}
max_len = []
def ntm_data(split_num=0,dtype='test'):

    
    f = open ('data-resplit/split.json', "r")  

    data = json.loads(f.read())[str(split_num)]
    r = data[dtype]
    pe,re=0,0
    f1 = 0
    num =0

    for file in r:
        num +=1
        dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
        tree = ET.parse(dir+file)
        root = tree.getroot()

        transcript, key, _, _= generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])

        sent_dict = defaultdict(str)
        flag = True
        group = 0
        
        all_drs = []
        predict_true,real_true,correct =0,0,0
        for content in root.findall('decisions'):
            for sentence in content.findall('sentence'):
                summary = sentence.text.replace("\n","")
                group += 1
                #f_name.append(file)
                decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                #print(decision_id)
                ext_link = link_map[decision_id]

                if ext_link ==[]:
                    flag = False
                else:
                    for i in ext_link:
                        left,right,_ = get_drda(file, i)
                        da,time = get_da(left,right)
                        das = sent_detector.tokenize(da)
                        for comp in das:
                            if 'just kinetic then' in comp:
                                comp = 'just kinetic then'
                            if len(comp.split())<3:
                                continue
                            pos, sentence, savedkey = search_trans(transcript, key, comp,time)
                            

                            if sentence =="":
                                print(comp)
                                continue

                            if len(sentence.split())<3:
                                continue

                            if pos not in sent_dict:
                                sentence_summary_map[sentence] = summary
                                all_drs.append(sentence[3:])
                                #tmp.append(sentence)
                            sent_dict[pos] = sentence

        for i in key:
            
            sentence = transcript[i][2:].lower()
            
            sentence = " ".join([i for i in sentence.split() if i not in utterance])

            if len(sentence.split())<3:
                continue
                

            predict= False
            real = False
            
            if sentence in all_drs:
                real = True       
                
            sentence = sentence.split()
            sentence = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else lemmatizer.lemmatize(i) for i,j in pos_tag(sentence)]

            tc = 0
            for word in sentence:
                if word in seed:
                    tc +=1
                    
            if tc>=1:
                predict = True

            if predict and real:
                correct +=1
            if predict:
                predict_true +=1
            if real:
                real_true += 1

        tmpp = correct*1.0/predict_true
        pe += tmpp
        
        tmpr= 0
        if real_true!=0:
            tmpr =  correct*1.0/real_true
        else:
            tmpr =1
        re += tmpr
        if tmpr+tmpp!=0:
            f1 += 2*tmpr*tmpp/(tmpr+tmpp)
        else:
            f1+=0
    print(pe/num)
    print(re/num)
    print(f1/num)
                 

    #print(true_count)
    #print(len(train_label))

ntm_data()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/heatherzheng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/heatherzheng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/heatherzheng/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/heatherzheng/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0.056082811722816615
0.6876011488511489
0.10016360883733014


In [165]:
#cluster train merge
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    all_sum = []
    f_name = []
    total_repeat = 0
    
    for cross_split in [1]:
        tfidf_input = []
        tfidf = None
        for mode in ["train",'val','test']:
            try:
                os.mkdir('merge_group_'+str(cross_split))
            except:
                pass
        
            try:
                os.mkdir('merge_group_'+str(cross_split)+"/"+mode)
            except:
                pass
            
                
            pos_example, neg_example = 0,0

            
            f = open ('data-resplit/split.json', "r")  

            data = json.loads(f.read())[str(cross_split)]
            r = data[mode]

            if tfidf_input == []:
                for file in data['train']:
                    _,_, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                    tfidf_input.extend(corpus)
                with open('merge_group_'+str(cross_split)+"/"+"tfidf.json", 'w') as f:
                    json.dump(tfidf_input, f)               
                vectorizer = TfidfVectorizer()
                tfidf = vectorizer.fit(tfidf_input)
                    
            for file in r:
                dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
                tree = ET.parse(dir+file)
                root = tree.getroot()

                
                transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                
                corpus_tfidf = tfidf.transform(corpus)
                textdict = {}
            
                flag = True
                group = 0
                
                drda = []
                stats = []
                
                sent_dict = defaultdict(str)
                group_info = defaultdict(set)
                group_to_sum = {}
                for content in root.findall('decisions'):
                    for sentence in content.findall('sentence'):
                        sent_dict = defaultdict(str)
                        
                        summary = sentence.text.replace("\n","")
                        
                        all_summaries.append(summary)
                        num_sent = 0
                        group += 1
                        group_to_sum[group] = summary
                        f_name.append(file)
                        decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                        ext_link = link_map[decision_id]
                        if ext_link ==[]:
                            flag = False
                        else:
                            for i in ext_link:
                                left,right,drda_type = get_drda(file, i)
                                da,time = get_da(left,right)
                                das = sent_detector.tokenize(da)
                                for comp in das:

                                    if 'just kinetic then' in comp:
                                        comp = 'just kinetic then'
                                    if len(comp.split())<3:
                                        continue
                                    if search_trans(transcript, key, comp,time)==None:
                                        #print(comp)
                                        #print(transcript)
                                        comp = comp.replace(". ","")

                                    pos, sentence, s_time = search_trans(transcript, key, comp,time)
                                    #print(s_time,time)

                                    if sentence =="":
                                        continue
                                    
                                    if pos not in sent_dict:
                                        
                                        if len(sentence.split())<3:
                                            continue
                                        context_s = corpus[find_context(tfidf.transform([sentence]).toarray(),corpus_tfidf.toarray())]
                                        #text.append((s_time,sentence,group,pos,context_s,summary))
                                        textdict[s_time] = [sentence,pos,context_s]
                                        stats.append(group)
                                        drda.append(sentence)
                                        num_sent +=1
                                        group_info[s_time].add(group)
                                    sent_dict[pos] = sentence
  
                        

                        avg.append(num_sent)

   
                merge_group = []
                for i in range(1,group+1):
                    merge_group.append([i])
                    
                    
                prev_group_info = group_info.copy()
                for i in prev_group_info:
                    if len(prev_group_info[i])>1:
                        
                        idx_pos = []
                        mlst = []
                        for j in prev_group_info[i]:
                            for gidx, g in enumerate(merge_group):
                                if j in g:
                                    idx_pos.append(gidx)
                                    mlst.extend(g)
                        new_merge_group = []
                        for idx,ele in enumerate(merge_group):
                            if idx not in idx_pos:
                                new_merge_group.append(ele)
                        merge_group = new_merge_group

                        merge_group.append(list(set(mlst)))

                text = []
                for k in textdict:
                    group_of_this_sent = list(group_info[k])
                    print(group_of_this_sent)
                    summary = " ".join([group_to_sum[i] for i in group_of_this_sent])
                    gnum =None
                    for idx,i in enumerate(merge_group):
                        if group_of_this_sent[0] in i:
                            gnum=idx
                    text.append([k,textdict[k][0],gnum,textdict[k][1],textdict[k][2],summary])
                    

                
                text.sort(key = lambda x: x[0]) 
                #pos/neg calculation
                c = list(dict(Counter(stats)).values())
                for i in range(len(c)):
                    if c[i]>1:
                        pos_example += c[i]*(c[i]-1)/2.0
                    for j in range(i+1,len(c)):
                        neg_example += c[i]*c[j]
                                       
                result = []
         
                for i in range(len(text)):
                    #sentence_pos,time, sentence, group, pos over passage, context sentence, drdas,summaries
                    result.append((i,text[i][0],text[i][1],text[i][2],text[i][3], text[i][4]," ".join(drda),text[i][5]))
                for i in text:
                    with open('merge_group_'+str(cross_split)+"/"+mode+"/"+file.split(".")[0]+".json", 'w') as f:
                        json.dump(result, f)
        
    print(total_repeat)
    print(sum(avg)/len(avg))
generate_data()

[1]
[1]
[1]
[1]
[1]
[1]
[2]
[2]
[2]
[2]
[2]
[2]
[1]
[2]
[3]
[1]
[1]
[1]
[1]
[1]
[2]
[3]
[3]
[4]
[5]
[5]
[5]
[5]
[5]
[1]
[2]
[2]
[2]
[2]
[2]


KeyboardInterrupt: 

In [ ]:
#neural_topic 
#for conflict change pair training algo:
#cluster train
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    all_sum = []
    f_name = []
    total_repeat = 0
    
    for cross_split in [2,3,4]:
        tfidf_input = []
        tfidf = None
        for mode in ["train",'val','test']:
            try:
                os.mkdir('fix_pair_algo_'+str(cross_split))
            except:
                pass
        
            try:
                os.mkdir('fix_pair_algo_'+str(cross_split)+"/"+mode)
            except:
                pass
            
                
            pos_example, neg_example = 0,0

            
            f = open ('data-resplit/split.json', "r")  

            data = json.loads(f.read())[str(cross_split)]
            r = data[mode]

            if tfidf_input == []:
                for file in data['train']:
                    _,_, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                    tfidf_input.extend(corpus)
                with open('fix_pair_algo_'+str(cross_split)+"/"+"tfidf.json", 'w') as f:
                    json.dump(tfidf_input, f)               
                vectorizer = TfidfVectorizer()
                tfidf = vectorizer.fit(tfidf_input)
                    
            for file in r:
                dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
                tree = ET.parse(dir+file)
                root = tree.getroot()

                
                transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                
                corpus_tfidf = tfidf.transform(corpus)
                textdict = {}
            
                flag = True
                group = 0
                
                drda = []
                stats = []
                
                sent_dict = defaultdict(str)
                group_info = defaultdict(set)
                group_to_sum = {}
                for content in root.findall('decisions'):
                    for sentence in content.findall('sentence'):
                        sent_dict = defaultdict(str)
                        
                        summary = sentence.text.replace("\n","")
                        
                        all_summaries.append(summary)
                        num_sent = 0
                        group += 1
                        group_to_sum[group] = summary
                        f_name.append(file)
                        decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                        ext_link = link_map[decision_id]
                        if ext_link ==[]:
                            flag = False
                        else:
                            for i in ext_link:
                                left,right,drda_type = get_drda(file, i)
                                da,time = get_da(left,right)
                                das = sent_detector.tokenize(da)
                                for comp in das:

                                    if 'just kinetic then' in comp:
                                        comp = 'just kinetic then'
                                    if len(comp.split())<3:
                                        continue
                                    if search_trans(transcript, key, comp,time)==None:
                                        #print(comp)
                                        #print(transcript)
                                        comp = comp.replace(". ","")

                                    pos, sentence, s_time = search_trans(transcript, key, comp,time)
                                    #print(s_time,time)

                                    if sentence =="":
                                        continue
                                    
                                    if pos not in sent_dict:
                                        
                                        if len(sentence.split())<3:
                                            continue
                                        context_s = corpus[find_context(tfidf.transform([sentence]).toarray(),corpus_tfidf.toarray())]
                                        #text.append((s_time,sentence,group,pos,context_s,summary))
                                        textdict[s_time] = [sentence,pos,context_s]
                                        stats.append(group)
                                        drda.append(sentence)
                                        num_sent +=1
                                        group_info[s_time].add(group)
                                    sent_dict[pos] = sentence
  
                        

                        avg.append(num_sent)
                text = []
                for k in group_info:
                    thissum = " ".join([group_to_sum[i] for i in group_info[k]])
                    text.append([k,textdict[k][0],list(group_info[k]),textdict[k][1],textdict[k][2],thissum])


                
                text.sort(key = lambda x: x[0]) 
                #pos/neg calculation
                c = list(dict(Counter(stats)).values())
                for i in range(len(c)):
                    if c[i]>1:
                        pos_example += c[i]*(c[i]-1)/2.0
                    for j in range(i+1,len(c)):
                        neg_example += c[i]*c[j]
                                       
                result = []
         
                for i in range(len(text)):
                    #sentence_pos,time, sentence, group, pos over passage, context sentence, drdas,summaries
                    result.append((i,text[i][0],text[i][1],text[i][2],text[i][3], text[i][4]," ".join(drda),text[i][5]))
                for i in text:
                    with open('fix_pair_algo_'+str(cross_split)+"/"+mode+"/"+file.split(".")[0]+".json", 'w') as f:
                        json.dump(result, f)
        
    print(total_repeat)
    print(sum(avg)/len(avg))
generate_data()

In [54]:
#cluster test

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    all_sum = []
    f_name = []
    

    
    for cross_split in [3,4]:

        file = open("/Users/heatherzheng/Downloads/word_"+str(cross_split)+".json")
        seed=json.load(file)
        print(seed)
        tfidf_input = []
        tfidf = None
        for mode in ['test','train','val']:
            try:
                os.mkdir('pair_sentence_filter_'+str(cross_split))
            except:
                pass
        
            try:
                os.mkdir('pair_sentence_filter_'+str(cross_split)+"/"+mode)
            except:
                pass
            
                
            pos_example, neg_example = 0,0

            
            f = open ('data-resplit/split.json', "r")  

            data = json.loads(f.read())[str(cross_split)]
            r = data[mode]


            if tfidf_input == []:
                for file in data['train']:
                    _,_, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                    tfidf_input.extend(corpus)
                with open('pair_sentence_filter_'+str(cross_split)+"/"+"tfidf.json", 'w') as f:
                    json.dump(tfidf_input, f)               
                vectorizer = TfidfVectorizer()
                tfidf = vectorizer.fit(tfidf_input)
                    
            for file in r:
                dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
                tree = ET.parse(dir+file)
                root = tree.getroot()

                
                transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])
                
                corpus_tfidf = tfidf.transform(corpus)
                text = []
            
                flag = True
                group = 0
                
                drda = []
                stats = []
                inputs = []
    
                for content in root.findall('decisions'):
                    for sentence in content.findall('sentence'):
                        summary = sentence.text.replace("\n","")
                        decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                        ext_link = link_map[decision_id]
                        tmp = []
                        if ext_link ==[]:
                            flag = False
                        else:
                            for i in ext_link:
                                left,right,drda_type = get_drda(file, i)
                                da,time = get_da(left,right)
                                das = sent_detector.tokenize(da)
                                for comp in das:
                                    if len(comp.split())<3:
                                        continue
                                    if 'just kinetic then' in comp:
                                        comp = 'just kinetic then'
                                    if search_trans(transcript, key, comp,time)==None:
                                        #print(comp)
                                        #print(transcript)
                                        comp = comp.replace(". ","")


                                    pos, sentence, s_time = search_trans(transcript, key, comp,time)
                                    tmp.append([s_time,summary])
                             
                        
                     
                        inputs.append(tmp)  
                      
                count = 0
                sum_map = {}
                for i in inputs:
                    for j in i:
                        sum_map[j[0]]=[count,j[1]]
                    count +=1
                    
                
                for index,k in enumerate(key):
                    ori_sent = [i.lower() for i in transcript[k][2:].split() if i.lower() not in utterance]
                    sent = " ".join(ori_sent)
                    
                    sentence = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else lemmatizer.lemmatize(i) for i,j in pos_tag(ori_sent)]

                    select =False
                    for word in sentence:
                        if word in seed:
                            select=True
                    if select:
                        context_s = corpus[find_context(tfidf.transform([sent]).toarray(),corpus_tfidf.toarray())]
                        if k in sum_map:
                            text.append((k,sent,sum_map[k][0],index,context_s,sum_map[k][1]))
                        else:
                            text.append((k,sent,count,index,context_s,""))
                            
                text.sort(key = lambda x: x[0]) 
                
                result = []
                for i in range(len(text)):
                    #sentence_pos,time, sentence, group, pos over passage, context sentence, drdas,summaries
                    result.append((i,text[i][0],text[i][1],text[i][2],text[i][3], text[i][4]," ".join(drda),text[i][5]))
                for i in text:
                    with open('pair_sentence_filter_'+str(cross_split)+"/"+mode+"/"+file.split(".")[0]+".json", 'w') as f:
                        json.dump(result, f)


generate_data()

{'button': '11', 'channel': '5', 'group': '25', 'euro': '2', 'number': '19', 'scroll': '5', 'price': '7', 'function': '14', 'twenty': '7', 'point': '17', 'fifty': '10', 'changeable': '20', 'display': '6', 'base': '12', 'selling': '7', 'cost': '2', 'rubber': '8', 'profit': '16', 'wheel': '5', 'logo': '4', 'design': '13', 'yellow': '15', 'device': '22', 'prototype': '16', 'corporate': '24', 'production': '2', 'battery': '0', 'user': '3', 'television': '29', 'twelve': '10', 'kinetic': '8', 'lcd': '8', 'recognition': '23', 'screen': '18', 'light': '9', 'menu': '21', 'volume': '26', 'colorful': '20', 'organic': '15', 'projected': '17', 'rechargeable': '0', 'station': '12', 'gray': '9', 'expert': '6', 'voice': '23', 'cutout': '4', 'separately': '19', 'marketing': '6', 'company': '4', 'shape': '15', 'advanced': '14', 'center': '11', 'aim': '16', 'order': '19', 'cover': '20', 'image': '24', 'speech': '23', 'control': '28', 'locator': '14', 'touch': '18', 'original': '27', 'product': '1', 'smal

In [56]:
from scipy import spatial
import bcubed
from math import log

def variation_of_information(X, Y):
  n = float(sum([len(x) for x in X]))
  sigma = 0.0
  for x in X:
    p = len(x) / n
    for y in Y:
      q = len(y) / n
      r = len(set(x) & set(y)) / n
      if r > 0.0:
        sigma += r * (log(r / p, 2) + log(r / q, 2))
  return abs(sigma)

def lst_to_pair(lst):
    pair = []
    for i in lst:
        if len(i)==1:
            continue

        for j in range(len(i)):
            for k in range(j+1, len(i)):
                pair.append((min(i[j],i[k]),max(i[j],i[k])))

    return pair

def get_pair_score(predict,expect):
    predict = lst_to_pair(predict)
    expect = lst_to_pair(expect)


    tp = 0
    for i in predict:
        if i in expect:
            tp +=1
    if expect:
        r = tp/len(expect)
    else:
        r = 1

    if predict:
        p = tp/len(predict)
    else:
        p= 1


    if p+r==0:
        return p,r,0

    return p,r,2*r*p/(r+p)


In [121]:
#summary by filter cluster
import bcubed
#f = open ('fix_4ntest_cluster.json', "r")
f = open ('/Users/heatherzheng/Desktop/Neural_Topic_Models/svm_4_cluster.json', "r")



cluster = json.loads(f.read())

r = set([i.split(".")[0]+".abssumm.xml" for i in cluster[1]])

bprecision,brecall,bfscore=0,0,0
voi = 0
err = 0
pairp,pairr,pairf = 0,0,0
for file in r:
    sents = []
    for i in range(len(cluster[0])):

        if file.split(".")[0] in cluster[1][i]:
            sents.append(cluster[0][i])
            

    
    dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
    tree = ET.parse(dir+file)
    root = tree.getroot()


    transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])

    text = []

    flag = True
    group = 0

    drda = []
    stats = []
    inputs = defaultdict(set)

    for content in root.findall('decisions'):
        for sentence in content.findall('sentence'):
            summary = sentence.text.replace("\n","")
            decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
            ext_link = link_map[decision_id]
            tmp = []
            if ext_link ==[]:
                flag = False
            else:
                for i in ext_link:
                    left,right,drda_type = get_drda(file, i)
                    da,time = get_da(left,right)
                    das = sent_detector.tokenize(da)
                    for comp in das:
                        if len(comp.split())<3:
                            continue
                        if 'just kinetic then' in comp:
                            comp = 'just kinetic then'
                        if search_trans(transcript, key, comp,time)==None:
                            #print(comp)
                            #print(transcript)
                            comp = comp.replace(". ","")


                        pos, sentence, s_time = search_trans(transcript, key, comp,time)
                        inputs[sentence[3:]].add(group)


            group +=1
    gold_group_num = group
    map_sent_to_idx = {}
    predict = defaultdict(set)
    count = 0
    for sent in enumerate(sents):
        usefulcluster=False
        for key in inputs:
            if key in sent[1]:
                if key not in map_sent_to_idx:
                    map_sent_to_idx[key] = len(map_sent_to_idx)+1
                predict[map_sent_to_idx[key]].add(count)
                usefulcluster = True
                

        if usefulcluster:
            count +=1
        
    gold = defaultdict(set)
    for i in inputs:
        if i in map_sent_to_idx:
            gold[map_sent_to_idx[i]] = inputs[i]
    gold = dict(gold)
    predict = dict(predict)
    
    if gold=={}:
        err += 1
        print(file)
        continue

    tmp_bp = bcubed.precision(predict, gold)

    bprecision += tmp_bp

    tmp_br = bcubed.recall(predict, gold)
    brecall += tmp_br
    
    bfscore += bcubed.fscore(tmp_bp,tmp_br)

    gold_lst = [[] for _ in range(gold_group_num)]

    for key in gold:
        for group in gold[key]:
            if key not in gold_lst[group]:
                gold_lst[group].append(key)
    predict_lst = [[] for _ in range(count)]
    for key in predict:
        for group in predict[key]:
            if key not in predict_lst[group]:
                predict_lst[group].append(key)
            
    gold_lst = [i for i in gold_lst if i!=[]]
    predict_lst = [i for i in predict_lst if i!=[]]

    pt,rt, ft= get_pair_score(predict_lst, gold_lst)

    pairp += pt
    pairr += rt
    pairf += ft
    voi += variation_of_information(predict_lst, gold_lst)
print(err)
print(bprecision/(len(r)-err),brecall/(len(r)-err),bfscore/(len(r)-err),
      pairp/(len(r)-err),pairr/(len(r)-err),pairf/(len(r)-err), voi/(len(r)-err))




TS3007b.abssumm.xml
1
0.6637577048496166 0.8712312675070029 0.7260919304069178 0.4221968731751341 0.8108745421245421 0.4588520351232216 1.1016034983908976


In [105]:
#pick balanced bad example
#summary by filter cluster
import bcubed

sec='test'
cross_split = '4'
f = open ('fix_'+cross_split+sec+'_cluster.json', "r")


cluster = json.loads(f.read())

r = set([i.split(".")[0]+".abssumm.xml" for i in cluster[1]])

pos_sample= []
neg_sample = []
for file in r:
    sents = []
    for i in range(len(cluster[0])):

        if file.split(".")[0] in cluster[1][i]:
            sents.append(cluster[0][i])
            

    
    dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
    tree = ET.parse(dir+file)
    root = tree.getroot()


    transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])

    text = []

    flag = True
    group = 0

    drda = []
    stats = []
    inputs = defaultdict(set)
    
    group_to_summary = {}
    for content in root.findall('decisions'):
        for sentence in content.findall('sentence'):
            summary = sentence.text.replace("\n","")
            group_to_summary[group]=summary
            decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
            ext_link = link_map[decision_id]
            tmp = []
            if ext_link ==[]:
                flag = False
            else:
                for i in ext_link:
                    left,right,drda_type = get_drda(file, i)
                    da,time = get_da(left,right)
                    das = sent_detector.tokenize(da)
                    for comp in das:
                        if len(comp.split())<3:
                            continue
                        if 'just kinetic then' in comp:
                            comp = 'just kinetic then'
                        if search_trans(transcript, key, comp,time)==None:
                            #print(comp)
                            #print(transcript)
                            comp = comp.replace(". ","")


                        pos, sentence, s_time = search_trans(transcript, key, comp,time)
                        inputs[sentence[3:]].add(group)


            group +=1


    for sent in enumerate(sents):
        cur_group = set()

        for key in inputs:
            if key in sent[1]:
                cur_group.update(inputs[key])

    
        if len(cur_group)>0:
            summ = []
            for g in cur_group:
                summ.append(group_to_summary[g])
            pos_sample.append([sent[1]," ".join(summ)])
        else:
            neg_sample.append([sent[1],"empty"])



In [106]:
import random
select_neg = random.sample(range(len(neg_sample)), 2*len(pos_sample))

result = []
for i in range(len(pos_sample)):
    result.append(pos_sample[i])
    
random.shuffle(result)

for i in range(len(result)):
    file1 = open("bart_from_step2_sum/cluster_"+cross_split+"_"+sec+".txt.src","a")
    file1.write(result[i][0]+"\n")
    file2 = open("bart_from_step2_sum/cluster_"+cross_split+"_"+sec+".txt.tgt.tagged","a")
    file2.write(result[i][1]+"\n")    
    
for i in select_neg:
    result.append(neg_sample[i])
random.shuffle(result)
for i in range(len(result)):
    file1 = open("bart_from_step2/cluster_"+cross_split+"_"+sec+".txt.src","a")
    file1.write(result[i][0]+"\n")
    file2 = open("bart_from_step2/cluster_"+cross_split+"_"+sec+".txt.tgt.tagged","a")
    file2.write(result[i][1]+"\n")  

In [162]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk import pos_tag

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()


def delete_rep(inputs):
    lst = []
    #topic = open("/home/sz488/Neural_Topic_Models/seed_1_filter_noverb_35.json")
    topic = open("/Users/heatherzheng/Downloads/seed_1_filter_noverb_35.json")

    seed_cluser_map = json.load(topic)
    for s in inputs:
        tmp = []
        s = word_tokenize(s)
        s_pos_tag = pos_tag(s)
        s = [lemmatizer.lemmatize(i,j[0].lower()) if j[0].lower() in ['a','n','v'] else lemmatizer.lemmatize(i) for i,j in s_pos_tag]
  
        for w in s:
            if w in seed_cluser_map:
                tmp.append(w.lower())
  
        lst.append(tmp)
    print(lst)

                
    all_word = set()
    sents = []
    remove = set()
    for i in range(len(lst)):
        max_rep = 0
        max_rep_sent = None
        for j in range(len(lst)):
            if i!=j:
                count = 0
                for word in lst[i]:
                    if word in lst[j]:
                        count +=1
                if count>max_rep:
                    max_rep = count
                    max_rep_sent = j
        if max_rep>len(lst[i])*0.9:
            if len(inputs[i].split())<len(inputs[max_rep_sent].split()) or (len(inputs[i].split())==len(inputs[max_rep_sent].split()) and i<max_rep_sent):
                remove.add(i)
    print(remove)
    ans = []
    for idx,sent in enumerate(inputs):
        if idx not in remove:
            ans.append(sent)
            
                
    
            
    return " ".join(ans)

KeyboardInterrupt: 

In [102]:
i = [" The remote will feature a locator function that responds to",
" The remote will feature a locator function that responds to"]

In [103]:
for s in i:
    print(s)
delete_rep(i)

 The remote will feature a locator function that responds to
 The remote will feature a locator function that responds to
[['feature', 'locator', 'function'], ['feature', 'locator', 'function']]
{0}


' The remote will feature a locator function that responds to'

In [60]:
#merge bart

#cluster train
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    all_sum = []
    f_name = []
    total_repeat = 0
    
    for cross_split in [1]:
        tfidf_input = []
        tfidf = None
        for mode in ["train"]:
      
            f = open ('data-resplit/split.json', "r")  

            data = json.loads(f.read())[str(cross_split)]
            r = data[mode]
            all_text = []
            all_summary = []
                    
            for file in r:
                dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
                tree = ET.parse(dir+file)
                root = tree.getroot()

                
                transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])

                textdict = {}
            
                flag = True
                group = 0
                
                drda = []
                stats = []
                
                sent_dict = defaultdict(str)
                group_info = defaultdict(set)
                group_to_sum = {}
                id_to_sent = {}
                group_to_ids = defaultdict(list)
                for content in root.findall('decisions'):
                    for sentence in content.findall('sentence'):
                        sent_dict = defaultdict(str)
                        
                        summary = sentence.text.replace("\n","")
                        
                        all_summaries.append(summary)
                        num_sent = 0
                        group += 1
                        group_to_sum[group] = summary
                        f_name.append(file)
                        decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
                        ext_link = link_map[decision_id]
                        if ext_link ==[]:
                            flag = False
                        else:
                            for i in ext_link:
                                left,right,drda_type = get_drda(file, i)
                                da,time = get_da(left,right)
                                das = sent_detector.tokenize(da)
                                for comp in das:

                                    if 'just kinetic then' in comp:
                                        comp = 'just kinetic then'
                                    if len(comp.split())<3:
                                        continue
                                    if search_trans(transcript, key, comp,time)==None:
                                        #print(comp)
                                        #print(transcript)
                                        comp = comp.replace(". ","")

                                    pos, sentence, s_time = search_trans(transcript, key, comp,time)
                                    #print(s_time,time)

                                    if sentence =="":
                                        continue
                                    
                                    if pos not in sent_dict:
                                        
                                        if len(sentence.split())<3:
                                            continue
                                        
                                        id_to_sent[s_time] = sentence
                                        group_to_ids[group].append(s_time)
                                        
                                    sent_dict[pos] = sentence
  
                        

                        avg.append(num_sent)

   
                merge_group = []
                for i in range(1,group+1):
                    merge_group.append([i])
                    
                    
                prev_group_info = group_info.copy()
                for i in prev_group_info:
                    if len(prev_group_info[i])>1:
                        
                        idx_pos = []
                        mlst = []
                        for j in prev_group_info[i]:
                            for gidx, g in enumerate(merge_group):
                                if j in g:
                                    idx_pos.append(gidx)
                                    mlst.extend(g)
                        new_merge_group = []
                        for idx,ele in enumerate(merge_group):
                            if idx not in idx_pos:
                                new_merge_group.append(ele)
                        merge_group = new_merge_group

                        merge_group.append(list(set(mlst)))


                
                for g in merge_group:
                    sent_group = set()
                    this_sum = []
                    ans = []
                    for i in g:
                        this_sum.append(group_to_sum[i])
                        for j in group_to_ids[i]:
                            sent_group.add(j)
                            

                    for j in sent_group:
                        ans.append(id_to_sent[j][3:])
                    all_text.append(" ".join(ans))
                    text_len.append(len(" ".join(ans).split()))
                    sum_len.append(len(" ".join(this_sum).split()))
                    all_summary.append(" ".join(this_sum))
            for i in range(len(all_summary)):
                file1 = open("bart_merge/cluster_"+str(cross_split)+"_"+mode+".txt.src","a")
                file1.write(all_text[i]+"\n")
                file2 = open("bart_merge/cluster_"+str(cross_split)+"_"+mode+".txt.tgt.tagged","a")
                file2.write(all_summary[i]+"\n")    

generate_data()

In [59]:
text_len = []
sum_len = []

In [63]:
max(sum_len)

42

In [168]:
#cluster test

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

all_summaries = []
def find_context(sent, corpus_tfidf):
    max_val = 0
    max_pos = None
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if val > max_val:
            max_val = val
            max_pos = i
            
    second_val = -0.1
    second_pos = None
    
    for i in range(len(corpus_tfidf)):
        val = cosine_similarity(sent,[corpus_tfidf[i]])[0][0]
        if i==max_pos:
            continue
        if val > second_val:
            second_val = val
            second_pos = i
        
    if second_pos ==None:
        print(second_val)
        print(len(corpus_tfidf))
    return second_pos

def generate_data():
    avg = []
    data = []
    all_sum = []
    f_name = []
    



    file = 'ES2005b.abssumm.xml'
    dir = '/Users/heatherzheng/Desktop/ami_public_manual_1.6.2/abstractive/'
    tree = ET.parse(dir+file)
    root = tree.getroot()
    transcript, key, corpus,_ = generate_transcript(files[file.split('.')[0]],speaker[file.split('.')[0]])

    text = []

    flag = True
    group = 0

    drda = []
    stats = []
    inputs = []
    sent_to_group=defaultdict(set)
    group_to_sum = defaultdict(str)
    for content in root.findall('decisions'):
        for sentence in content.findall('sentence'):
            summary = sentence.text.replace("\n","")
            group_to_sum[group]=summary
            decision_id = sentence.attrib['{http://nite.sourceforge.net/}id']
            ext_link = link_map[decision_id]
            tmp = []
            if ext_link ==[]:
                flag = False
            else:
                for i in ext_link:
                    left,right,drda_type = get_drda(file, i)
                    da,time = get_da(left,right)
                    das = sent_detector.tokenize(da)
                    for comp in das:
                        if len(comp.split())<3:
                            continue
                        if 'just kinetic then' in comp:
                            comp = 'just kinetic then'
                        if search_trans(transcript, key, comp,time)==None:
                            #print(comp)
                            #print(transcript)
                            comp = comp.replace(". ","")


                        pos, sentence, s_time = search_trans(transcript, key, comp,time)
                        sent_to_group[sentence[3:]].add(group)
                        tmp.append([s_time,summary])
            group += 1

    
    #print(sent_to_group)
    text = []

    for index,k in enumerate(key):
        ori_sent = [i.lower() for i in transcript[k][2:].split() if i.lower() not in utterance]

        sent = transcript[k][:2]+" "+" ".join(ori_sent)
        text.append([k,sent])


    text.sort(key = lambda x: x[0]) 
    
    skipline = 0

    all_prev = []
    for i in text:

        if i[1][3:] in sent_to_group:
            print(skipline)
            print("___")
            print(i[1])

            print(sent_to_group[i[1][3:]])
            print("___")
            skipline = 0
            
            all_prev = []
        else:
            skipline +=1
            all_prev.append(i[1])
            #print(i[1])
    print(skipline)
    print(group_to_sum)


generate_data()

305
___
C: , now , we had as listed options we had speech recognition potentially , flat screen interface , lcd interface we also want to limit the number of buttons so we'll pretty much take that one as read .
{1}
___
92
___
C: then you know they're gonna be fairly brightly coloured anyway , and we can have sort of a a a trimming as well , of the glow in the dark material , just as gimmickyness .
{2}
___
18
___
C: so we'll stick with sort of programmability for the buttons that we do have .
{0}
___
19
___
C: excellent , so we'll go with speech recognition , yeah ?
{0}
___
1
___
C: , speech recognition , limited buttons , organic design .
{0, 1, 2}
___
0
___
D: if it's not too expensive s i think it's a good gimmick .
{2}
___
1
___
B: glow in dark .
{2}
___
98
defaultdict(<class 'str'>, {0: 'The remote will feature speech recognition and programmable channel and volume preferences.', 1: 'The number of button functions will be limited to make the remote user-friendly.', 2: 'The remote w